# CSP(1 point)
In the following code, only two of the function usages are essential. One is `is_valid(self, state)`, which checks if the current state is legal; the other is `is_satisfy(self, state)`, which is to checks if the current board meets the win condition. 
The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.

In test block, for `Solver`,  `n` indicates the size of the test while `method` indicates which bts function will be used(`bts` or `improving_bts`or`min_conflict`); for method `solve`, `render` indicates whether to use a graphical interface representation.

## Question 1: You should complete the function `bts()`. (0.8 points)
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. You may add any function you want. 
**Remember to test the case where N=6**

## Question 2: You should complete the function `improving_bts()` or `min_conflict()`. (0.2 points)
For `improving_bts()`, you can use one or more of the following strategies: Minimum Remaining Value, Least constraining value, and Forward checking. You should have a good performance (within 2s) **when N = 16 without GUI**. 

For `min_conflict()`, you should have a good performance  (within 2s) **when N = 100 without GUI**.

### DDL: 22:00, Nov.1st
The practice will be checked in this lab class or the next lab class(before **Nov.1st**) by teachers or SAs. 
#### What will be tested: 
* That you understand every line of your code, not just copy from somewhere 
* That your program compiles correctly
* Correctness of the program logic 
* That the result is obtained in a reasonable time 

#### Grading: 
* Submissions in this lab class: 1.1 points.
* Submissions on time: 1 point.
* Late submissions within 2 weeks after the deadline: 0.8 points.


In [134]:
import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.N = n

    def is_valid(self, state):
        """
        check whether the state satisfies the condition or not.
        : param state: list of points (in (row id, col id) tuple form)
        : return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.N

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.N - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.N - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.N - i - 1, self.N - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.N, self.N], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.N + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.N + 1))


In [135]:
class Solver:
    def __init__(self, n, method):
        self.N = n
        self.problem = Problem(n)
        self.method = method

    def solve(self, render=True):
        if render:
            import pygame
            w, h = 90 * self.N + 10, 90 * self.N + 10
            screen = pygame.display.set_mode((w, h))
            screen.fill('white')
            action_generator = self.method(self.problem)
            clk = pygame.time.Clock()
            queen_img = pygame.image.load('./queen.png')
            while True:
                for event in pygame.event.get():
                    if event == pygame.QUIT:
                        exit()
                try:
                    actions = next(action_generator)
                    screen.fill('white')
                    for i in range(self.n + 1):
                        pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                        pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                    for action in actions:
                        i, j = action
                        screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                    pygame.display.flip()
                except StopIteration:
                    pass
                clk.tick(5)
            pass
        else:
            start_time = time.time()
            for actions in self.method(self.problem):
                pass
            self.problem.print_state(actions)
            print(time.time() - start_time)

# BTS: Backtracking search

In [136]:
def bts(problem):
    action_stack = []
    row, col = 0, 0
    n = problem.N
    while not problem.is_satisfy(action_stack):
        action_stack.append((row, col))
        yield action_stack
        while col < n and not problem.is_valid(action_stack):
            col += 1
            action_stack.pop()
            action_stack.append((row, col))
            yield action_stack
        if col < n:
            if row == n-1:
                yield action_stack
                return action_stack
            row += 1
            col = 0
        else:
            action_stack.pop()
            row -= 1
            col = action_stack.pop()[1] + 1


# Improving BTS 
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

## Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
## Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
## Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [137]:
def improving_bts(problem):
    action_stack = []
    pos = dict()
    n = problem.N
    for l in range(n):
        pos[l] = -1
    row, col = 0, 0
    while not problem.is_satisfy(action_stack):
        while col < n:
            if is_valid(n, pos, row, col):
                pos[row] = col
                row += 1
                col = 0
                break
            else:
                col += 1
        if col == n:
            if row == 0:
                return None
            pos[row] = -1
            row -= 1
            col = pos[row] + 1
        if row == n:
            for l in range(n):
                action_stack.append((l, pos[l]))
            #print(action_stack)
        yield action_stack


def is_valid(n, pos, row, col):
    flag = True
    for k in range(n):
        if row == k or pos[k] == -1:
            continue
        if col == pos[k]:
            flag = False
            break
        if abs(k - row) == abs(pos[k] - col):
            flag = False
            break
    return flag
        



## Local search for CSP(min-conflict algorithm)

* First generate a complete assignment for all variables (this set of assignments may conflict)

* Repeat the following steps until there are no conflicts:

    * Randomly Select a variable that causes conflicts

    * Reassign the value of this variable to another value that with the least constraint onflicts with other variables

In [138]:
import random
def min_conflict(problem):
    n = problem.N
    state = [(i, random.randint(0, n - 1)) for i in range(n)]
    action_stack = []
    max = 500
    
    def conflicts(row, col):
        conflict_count = 0
        for r, c in state:
            if r != row:
                if c == col or abs(row - r) == abs(col - c):
                    conflict_count += 1
        return conflict_count

    for _ in range(max):
        if problem.is_satisfy(state):
            yield action_stack
            return action_stack
        conflicted_queens = []
        for r,c in state:
            if conflicts(r, c) > 0:
                conflicted_queens.append((r, c))
        
        row, _ = random.choice(conflicted_queens)
        min_conflict_col = min(
            range(n), key=lambda col: conflicts(row, col)
        )
        state[row] = (row, min_conflict_col)
        yield state
    return None

## test block

In [139]:
Solver(n=6, method=bts).solve(render=False)

_____________
|·|Q|·|·|·|·|
|·|·|·|Q|·|·|
|·|·|·|·|·|Q|
|Q|·|·|·|·|·|
|·|·|Q|·|·|·|
|·|·|·|·|Q|·|
-------------
0.013412237167358398


In [140]:
Solver(n=16, method=improving_bts).solve(render=False)

_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
---------------------------------
0.16969013214111328


In [141]:
Solver(n=100, method=min_conflict).solve(render=False)

_________________________________________________________________________________________________________________________________________________________________________________________________________
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·